In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.express as px
from backend.app.fetch_data_yf import CSV, data_dir

In [ ]:
class CryptoEvaluationPipeline(object):
    def __init__(self, data_file: str, clustered_data_file: str, representative_coins: list, window_days=[1, 7, 10]):
        """
        Initialise the evaluation pipeline.
        
        Args:
            data_file (str) : path the the top 30 cryptos dataset
            clustered_data_file (str) : path to the PCA clustered dataset
            windw_days (list): time horizon in days
        """
        self.data_df: pd.DataFrame = pd.read_csv(data_file, sep=',', encoding='utf8', index_col=0, parse_dates=True)
        self.clustered_data_file: pd.DataFrame = pd.read_csv(clustered_data_file, sep=',', encoding='utf8', index_col=0, parse_dates=True)
        self.representative_coins: dict = {}
        self.time_horizon: list = window_days
        self.num_pos_neg_corr_coins = 4
        # create a mapping of clusters and the representative coins
        for coin in representative_coins:
            cluster = self.clustered_data_file.at[coin, 'cluster']
            self.representative_coins[cluster] = coin
        
        print(self.representative_coins)
        
        # build the correlatino map, from representative coins, for analysis
        self.correlation_map = self._build_correlation_map()
        
    def _analyse_correlations(self):
        """
        Analyse correlations between selected coins and all cryptos in the dataset
        
        Args
            data (pandas.DataFrame): dataset of all coins and their closing prices
            crypto_cluster_representatives (list):
                                    list of selected coins
        Returns
            DataFrame: Correlation matrix
        """
        # get column names for representatives - should match but best to extract from the dataset
        representative_col_names = [col for col in self.data_df.columns if any(rep.lower() in col.lower() for rep in self.representative_coins.values())]
        print("representative coins found in dataset:\n",representative_col_names)
        # calculate the correlation matrix for the entire close dataset
        corr_matrix = self.data_df.corr()
        # extract the correlations for the selected coins
        representatives_correlation_matrix = corr_matrix[representative_col_names]
        # plot correlation heatmap
        # fig = px.imshow(
        #     representatives_correlation_matrix,
        #     labels=dict(color="correlation"),
        #     color_continuous_scale="Inferno_r",
        #     text_auto=True,
        #     aspect='auto',
        #     height=1000,
        # )
        # fig.update_xaxes(side='top')
        # fig.show()
        return representatives_correlation_matrix
        
    def _build_correlation_map(self):
        results = {}
        corr_matrix = self._analyse_correlations()
        # iterate through each column in the matrix
        for repr_col in corr_matrix.columns:
            # get the correlation for this column, dropping the self-corr column
            representative_corr = corr_matrix[repr_col].drop(repr_col)
            # get the list of the top 4 positive correlations
            top_positives: pd.Series = representative_corr.nlargest(self.num_pos_neg_corr_coins)
            # get the list of the top 4 negative correlations
            top_negatives: pd.Series = representative_corr.nsmallest(self.num_pos_neg_corr_coins)
            # add to dict, a dict of the postive and negative corrs for a given column (coin)
            repr_name = repr_col.split('-')[0]
            results[repr_name] = {
                'positive': [(col.split('-')[0], corr) for col, corr in top_positives.items()],
                'negative': [(col.split('-')[0], corr) for col, corr in top_negatives.items()]
            }
        # TODO markdown this neatly into a table
        for coin, data in results.items():
            print("%s - top positively correlated coins:" % coin)
            for corr_pos_coin, corr in data['positive']:
                print(f"Positively correlated coin: {corr_pos_coin} - Correlation: {corr}")
            print("%s - top negatively correlated coins:" % coin)
            for corr_neg_coin, corr in data['negative']:
                print(f"Negatively correlated coin: {corr_neg_coin} - Correlation: {corr}")
        # return the dict
        return results
        
foo = CryptoEvaluationPipeline(CSV, data_dir+"/top_30_cryptos_past_year_pca_cluster.csv", ['BNB-USD', 'BTC-USD', 'WETH-USD'])
        
        

{0: 'BNB-USD', 1: 'BTC-USD', 2: 'WETH-USD'}
representative coins found in dataset:
 ['BNB-USD', 'BTC-USD', 'WBTC-USD', 'WETH-USD']
BNB - top positively correlated coins:
Positively correlated coin: BTC - Correlation: 0.8004668464364069
Positively correlated coin: WBTC - Correlation: 0.8000557892127584
Positively correlated coin: DOGE - Correlation: 0.7662133024674067
Positively correlated coin: LEO - Correlation: 0.7652108206375713
BNB - top negatively correlated coins:
Negatively correlated coin: TON - Correlation: -0.22135586874787905
Negatively correlated coin: USDT - Correlation: -0.16777666016087872
Negatively correlated coin: USDC - Correlation: 0.04091938808269733
Negatively correlated coin: DOT - Correlation: 0.20274037066896677
BTC - top positively correlated coins:
Positively correlated coin: WBTC - Correlation: 0.9999705446700913
Positively correlated coin: LEO - Correlation: 0.9586002839886542
Positively correlated coin: DOGE - Correlation: 0.9365443127151238
Positively cor

/var/folders/25/dm1wkhps30vcw00tp01rzgt80000gn/T/ipykernel_95938/3089553341.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.clustered_data_file: pd.DataFrame = pd.read_csv(clustered_data_file, sep=',', encoding='utf8', index_col=0, parse_dates=True)
